In [1]:
import tensorflow as tf
import numpy as np

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sentence = " if you want you"

In [3]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

In [4]:
dataX = []
dataY = []

seq_length = 10

for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i+seq_length]
    y_str = sentence[i + 1: i + seq_length + 1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)

0  if you wa -> if you wan
1 if you wan -> f you want
2 f you want ->  you want 
3  you want  -> you want y
4 you want y -> ou want yo
5 ou want yo -> u want you


In [5]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)

batch_size = len(dataX)

In [6]:
X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

In [7]:
X_one_hot = tf.one_hot(X, num_classes)

In [8]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)
initial_state = cell.zero_state(batch_size, tf.float32)

In [9]:
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state = initial_state, dtype = tf.float32)

weights = tf.ones([batch_size, seq_length])
seq_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, weights = weights)
train = tf.train.AdamOptimizer(learning_rate = 0.1).minimize(seq_loss)

In [10]:
prediction = tf.argmax(outputs, axis = 2)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
l, _ = sess.run([seq_loss, train], feed_dict = {X: dataX, Y: dataY})
result = sess.run(prediction, feed_dict={X:dataX})

In [26]:
result

array([[0, 3, 3, 3, 3, 3, 0, 0, 0, 0],
       [5, 3, 3, 3, 0, 0, 0, 0, 9, 9],
       [5, 3, 3, 9, 0, 0, 0, 9, 9, 9],
       [0, 9, 0, 0, 0, 0, 9, 9, 9, 9],
       [9, 9, 9, 0, 0, 9, 9, 9, 9, 9],
       [4, 9, 0, 0, 9, 9, 9, 9, 9, 9]], dtype=int64)

In [27]:
result.reshape(-1)

array([0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 5, 3, 3, 3, 0, 0, 0, 0, 9, 9, 5, 3,
       3, 9, 0, 0, 0, 9, 9, 9, 0, 9, 0, 0, 0, 0, 9, 9, 9, 9, 9, 9, 9, 0,
       0, 9, 9, 9, 9, 9, 4, 9, 0, 0, 9, 9, 9, 9, 9, 9], dtype=int64)

In [28]:
result_str = [char_set[c] for c in result.reshape(-1)]

In [29]:
for i in range(3000):
    l, _ = sess.run([seq_loss, train], feed_dict = {X: dataX, Y: dataY})
    result = sess.run(prediction, feed_dict={X:dataX})
    
    result_str = [char_set[c] for c in np.squeeze(result)]
    print(i, "loss: ", l, "prediction: ", ''.join(result_str))

TypeError: only integer scalar arrays can be converted to a scalar index